In [1]:
from hashlib import sha1, pbkdf2_hmac
import hmac
import sys

In [18]:
def encrypt1(var, key):
    return bytes(a ^ b for a, b in zip(var, key))
#The second one uses int.from_bytes and int.to_bytes: 20 x faster
def encrypt2(var, key):
    key = key[:len(var)]
    int_var = int.from_bytes(var, sys.byteorder)
    int_key = int.from_bytes(key, sys.byteorder)
    int_enc = int_var ^ int_key
    return int_enc.to_bytes(len(var), sys.byteorder)
toHex = lambda x:"".join([hex(ord(c))[2:].zfill(2) for c in x])  #return string

# initiate i_pad and o_pad
i_pad = b'6'*64
o_pad = b'\\'*64

def hmac_sha1(key,msg):
    print('key=',key,'msg=',msg)
    
    key = key+b'\0'*(64-len(key))
    
    #key = key.ljust(blocksize, b'\0')
    print(key,len(key))

    i_key_pad = encrypt2(i_pad, key)
    print(i_key_pad)

    o_key_pad = encrypt2(o_pad, key)
    print(o_key_pad)

    #first pass
    print('First pass')
    first_msg = i_key_pad + msg
    print(first_msg, len(first_msg))
    hash1 = sha1(first_msg)
    #print(hash1.hexdigest())
    print(hash1.digest())

    #second pass
    print('Second pass')
    second_msg = o_key_pad + hash1.digest()
    print(second_msg, len(second_msg))
    hash2 = sha1(second_msg)
    #print(hash2.hexdigest())  
    return hash2.hexdigest()
    
def test1():   
    hash=hmac_sha1(b'12345',b'abcd')
    print('computed hmac',hash)
    #dk = pbkdf2_hmac('sha1', b'abcd', b'12345', 100000)  # pbkdf : password based key derivation function
    #print('by pbkdf2-hmac : ',dk.hex())
    value = hmac.new(b"12345", b"abcd",sha1).digest().hex()
    print(value)
    
def test2():
    hash = hmac_sha1(b"key", b"The quick brown fox jumps over the lazy dog")
    print('computed hmac',hash)
    #print('value must be de7c9b85b8b78aa6bc8a7a36f70a90701c9db4d9')
    value = hmac.new(b"key", b"The quick brown fox jumps over the lazy dog",sha1).digest().hex()
    print(value)
    
def test3():
    dk = pbkdf2_hmac('sha1', b"The quick brown fox jumps over the lazy dog", b"key",  100000)
    print('by pbkdf2-hmac : ',dk.hex())

In [19]:
test1()

key= b'12345' msg= b'abcd'
b'12345\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' 64
b'\x07\x04\x05\x02\x0366666666666666666666666666666666666666666666666666666666666'
b'mnohi\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'
First pass
b'\x07\x04\x05\x02\x0366666666666666666666666666666666666666666666666666666666666abcd' 68
b'\x0eUM\x0f\xa2\xea\x12\xc0\xfa\xed\xef\xad\xd2Cw\xdb\r\xef\xadV'
Second pass
b'mnohi\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\x0eUM\x0f\xa2\xea\x12\xc0\xfa\xed\xef\xad\xd2Cw\xdb\r\xef\xadV' 84
computed hmac 95bf7709dfeb0af2c6567398571fc0d20fa1e78a
95bf7709dfeb0af2c6567398571fc0d20fa1e78a


In [20]:
test2()

key= b'key' msg= b'The quick brown fox jumps over the lazy dog'
b'key\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' 64
b']SO6666666666666666666666666666666666666666666666666666666666666'
b'79%\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\'
First pass
b']SO6666666666666666666666666666666666666666666666666666666666666The quick brown fox jumps over the lazy dog' 107
b'\x0e\xfe\x15\xf9\x16\x1a\xb7(\xa2o\x1d\xc32\xf2f\x99\x13\xdcP~'
Second pass
b'79%\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\x0e\xfe\x15\xf9\x16\x1a\xb7(\xa2o\x1d\xc32\xf2f\x99\x13\xdcP~' 84
computed hmac de7c9b85b8b78aa6bc8a7a36f70a90701c9db4d9
de7c9b85b8b78aa6bc8a7a36f70a9070

In [17]:
test3()

by pbkdf2-hmac :  16fd8ff92f15a8059ce897335138a4a987c1e1e2
